In [0]:
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df_train = pd.read_csv('/content/gdrive/My Drive/adr-extraction-master/task3_training.tsv',sep = "\t", encoding = "utf-8-sig")
df_test = pd.read_csv('/content/gdrive/My Drive/adr-extraction-master/task3_validation.tsv',sep = "\t", encoding = "utf-8-sig")

In [0]:
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]

In [1]:
df_train = df_train.dropna(subset=['type'])
df_train = df_train.groupby('tweet_id',sort=False).agg(lambda x: x.tolist())

NameError: ignored

In [0]:
def merge(intervals):
  if len(intervals)==0:
      return []
  intervals = sorted(intervals,key = lambda x : x[0])
  current = intervals[0]
  result = []
  for i in range(len(intervals)):
      if current[0]<=intervals[i][0]<=current[1]:
          current[1] = max(current[1],intervals[i][1])
          continue
      else:
          result.append(current)
          current = intervals[i]
  result.append(current)
  return result

In [0]:
df_train.head()

,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
tweet_id,,,,,,,,
326398829849092097,[32.0],[43.0],[ADR],[gain weight],[fluoxetine],[i think my tablets have made me gain weight. ...,[10047896.0],[weight gain]
326462255468277761,[67.0],[73.0],[ADR],[dreams],[quetiapine],[suffice to say i am not particularly a fan of...,[10013632.0],[dreams abnormal]
326614974128611328,[8.0],[26.0],[ADR],[light nights sleep],[venlafaxine],[another light nights sleep brought to me cour...,[10040995.0],[sleep disturbance]
326665428493557760,[40.0],[53.0],[ADR],[added 1 stone],[quetiapine],[@pseudodeviant i clock in at 16 stone (added ...,[10047896.0],[weight gain]
326732768149135360,"[16.0, 38.0, 121.0]","[27.0, 47.0, 131.0]","[ADR, ADR, ADR]","[limbs ached, shivering, withdrawal]","[venlafaxine, venlafaxine, venlafaxine]",[wondered why my limbs ached and i was shiveri...,"[10000448.0, 10040558.0, 10073281.0]","[aching in limb, shivering, withdrawal symptom]"


In [0]:
df_train.to_csv ('/content/gdrive/My Drive/adr-extraction-master/task3_edited.csv', index = False, header=True)

In [0]:
import math

TRAIN_DATA = []
for index,row in df_train.iterrows():
  entityArray = []
  for i in range(len(row.begin)):
      entityArray.append([int(row.begin[i]),int(row.end[i])])
  entityArray = merge(entityArray)
  new = []
  for item in entityArray:
    new.append((item[0],item[1],"ADR"))
  TRAIN_DATA.append((row.tweet[0],{"entities": new}))

TRAIN_DATA[0]
    
    

('i think my tablets have made me gain weight. anyone on fluoxetine/prozac? #replytweet',
 {'entities': [(32, 43, 'ADR')]})

In [0]:
final = []
# with open('/content/gdrive/My Drive/adr-extraction-master/task2_combined.csv', 'w') as f:
for item in TRAIN_DATA:
  begin = []
  end = []
  for i in range(len(item[1]["entities"])):
    begin.append(str(item[1]["entities"][i][0]))
    end.append(str(item[1]["entities"][i][1]))
  final.append([item[0],",".join(begin),",".join(end)])
final[0]
      # f.write("{}{}{}{}".format(item[0],begin,end,'\n'))

['i think my tablets have made me gain weight. anyone on fluoxetine/prozac? #replytweet',
 '32',
 '43']

In [0]:
import csv

with open("/content/gdrive/My Drive/adr-extraction-master/task2_training.csv", "w", newline="",encoding = "utf-8") as f:
    writer = csv.writer(f,delimiter='\t')
    writer.writerows(final)

In [0]:
def main(model=None, output_dir="/content/gdrive/My Drive/adr-extraction-master", n_iter=100):
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == "__main__":
    main()



In [0]:
df_test.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death


In [0]:
df_test = df_test.dropna(subset=['type'])

In [0]:
df_test = df_test.groupby('tweet_id').agg(lambda x: x.tolist())

In [0]:
df_test.head()

,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
tweet_id,,,,,,,,
326406322323066883,"[111.0, 45.0]","[117.0, 71.0]","[ADR, ADR]","[zombie, feel like crying but can't]","[olanzapine, olanzapine]",[thought of work is overwhelming me so much i ...,"[10016384.0, 10011469.0]","[felt like a zombie, crying]"
326455480891346947,"[108.0, 46.0]","[122.0, 55.0]","[ADR, ADR]","[hot &amp; cold, Lethargic]","[venlafaxine, venlafaxine]",[5hrs sleep last night but stayed awake today....,"[10016335.0, 10024262.0]","[feeling hot and cold, lethargic]"
326575463835250689,[51.0],[56.0],[ADR],[awake],[quetiapine],[@rbhu7_82 if i don't take my quetiapine i can...,[10041014.0],[sleepiness]
326596903791902720,"[36.0, 52.0]","[41.0, 77.0]","[ADR, ADR]","[sleep, stomach is a cement mixer]","[ofloxacin, ofloxacin]",[ciprofloxacin: how do you expect to sleep whe...,"[10041017.0, 10042112.0]","[sleeplessness, stomach pain]"
326789014290055168,"[26.0, 53.0]","[32.0, 75.0]","[ADR, ADR]","[hungry, want to eat my own arm]","[olanzapine, olanzapine]",[@dopamine_diva q makes me hungry. olanzapine ...,"[10020466.0, 10020466.0]","[hunger, hunger]"


In [0]:
df_test.to_csv ('/content/gdrive/My Drive/adr-extraction-master/task3_edited.csv', index = False, header=True)

In [0]:
# TEST_DATA = []
# for index,row in df_test.iterrows():
#   TEST_DATA.append((row.tweet,{"entities":[(int(row.begin),int(row.end),"ADR")]}))

# TEST_DATA[1]

TypeError: ignored

In [0]:
nlp2 = spacy.load("/content/gdrive/My Drive/adr-extraction-master")
# for text, _ in TEST_DATA:
#     doc = nlp2(text)
#     print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#     print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [0]:
import math

TEST_DATA = []
for index,row in df_test.iterrows():
  entityArray = []
  for i in range(len(row.begin)):
      entityArray.append([int(row.begin[i]),int(row.end[i])])
  entityArray = merge(entityArray)
  new = []
  for item in entityArray:
    new.append((item[0],item[1],"ADR"))
  TEST_DATA.append((row.tweet[0],{"entities": new}))

TEST_DATA[0]
    
    

("thought of work is overwhelming me so much i feel like crying but can't because olanzapine has me trapped in a zombie state #strugglewithin",
 {'entities': [(45, 71, 'ADR'), (111, 117, 'ADR')]})

In [0]:
final = []
# with open('/content/gdrive/My Drive/adr-extraction-master/task2_combined.csv', 'w') as f:
for item in TEST_DATA:
  begin = []
  end = []
  for i in range(len(item[1]["entities"])):
    begin.append(str(item[1]["entities"][i][0]))
    end.append(str(item[1]["entities"][i][1]))
  final.append([item[0],",".join(begin),",".join(end)])
final[0]
      # f.write("{}{}{}{}".format(item[0],begin,end,'\n'))

["thought of work is overwhelming me so much i feel like crying but can't because olanzapine has me trapped in a zombie state #strugglewithin",
 '45,111',
 '71,117']

In [0]:
df_test.to_csv ('/content/gdrive/My Drive/adr-extraction-master/task3_edited.csv', index = False, header=True)

In [0]:
import csv

with open("/content/gdrive/My Drive/adr-extraction-master/task2_training.csv", "w", newline="",encoding = "utf-8-sig") as f:
    writer = csv.writer(f,delimiter='\t')
    writer.writerows(final)

In [0]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot["entities"])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

# examples = [
#     ('Who is Shaka Khan?',
#      [(7, 17, 'PERSON')]),
#     ('I like London and Berlin.',
#      [(7, 13, 'LOC'), (18, 24, 'LOC')])
# ]
 # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(nlp2, TEST_DATA)

In [0]:
print (results)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 53.103448275862064, 'ents_r': 33.771929824561404, 'ents_f': 41.286863270777474, 'ents_per_type': {'ADR': {'p': 53.103448275862064, 'r': 33.771929824561404, 'f': 41.286863270777474}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}
